# Process Simulation & Modeling for Chemical Engineers: Simulating a CSTR

**Objective:** This lesson will guide you through the fundamental steps of building a dynamic process model for a common piece of chemical equipment: the Continuous Stirred-Tank Reactor (CSTR). 

**Learning Goals:** By the end of this notebook, you will understand:
1.  How to apply the principles of material balance to a reacting system.
2.  How to translate fundamental physics (material balances, reaction kinetics) into a system of Ordinary Differential Equations (ODEs).
3.  How to use standard Python scientific libraries (`NumPy`, `SciPy`, `Matplotlib`) to solve these ODEs numerically.
4.  How to visualize and interpret the simulation results to understand the dynamic behavior of a chemical process.

## Part 1: Defining the System - A Continuous Stirred-Tank Reactor (CSTR)

A CSTR is a workhorse of the chemical industry. It's essentially a tank equipped with an impeller to ensure its contents are well-mixed. A reactant stream flows in, and a product stream flows out continuously.

![CSTR Diagram](https://upload.wikimedia.org/wikipedia/commons/e/e0/Cstr.png)

**Key Assumptions for our Model:**
*   **Perfect Mixing:** The impeller is so effective that the concentration and temperature are uniform throughout the entire tank. This means the properties of the outlet stream are identical to the properties of the fluid inside the tank.
*   **Constant Volume & Density:** The liquid-phase reaction does not cause a significant change in density, and a level controller keeps the volume in the tank constant. Therefore, the volumetric flow rate in ($F_{in}$) equals the volumetric flow rate out ($F_{out}$).

**The Reaction:**
We will model a simple, irreversible, first-order, liquid-phase reaction:
$$ A \rightarrow B $$
The rate of reaction ($r$) is given by the rate law: **$r = k C_A$**
*   $r$ = rate of consumption of A (moles per liter per minute)
*   $k$ = the reaction rate constant (1/min)
*   $C_A$ = the concentration of reactant A in the reactor (mol/L)

**Our Goal:** To simulate what happens to the concentrations of A and B inside the reactor from the moment we start feeding it ($t=0$) until it reaches a stable operating condition (steady state).

## Part 2: Preparing Our Tools & Defining Parameters

First, we need to import the Python libraries that will do the heavy lifting for us and define the parameters of our specific system.

In [ ]:
# Import necessary libraries
import numpy as np  # For numerical operations and arrays
from scipy.integrate import solve_ivp  # The ODE solver we will use
import matplotlib.pyplot as plt  # For plotting our results

In [ ]:
# --- Define the Process Parameters (Givens) ---

# Reactor & Flow Conditions
V = 1000.0        # Volume of the reactor (L)
flow_rate = 10.0  # Volumetric flow rate in and out (L/min)
C_A_in = 1.0      # Concentration of A in the inlet stream (mol/L)
C_B_in = 0.0      # Concentration of B in the inlet stream (mol/L)

# Reaction Kinetics
k = 0.1           # Reaction rate constant (1/min)

# --- Initial Conditions ---
# At time t=0, the reactor is filled with fluid of the same concentration as the inlet.
C_A0 = 1.0        # Initial concentration of A in the reactor (mol/L)
C_B0 = 0.0        # Initial concentration of B in the reactor (mol/L)

print("All parameters and initial conditions are defined.")

## Part 3: The Physics - Writing the Material Balances

The foundation of all process modeling is the law of conservation of mass. For any component in our system, we can write a general material balance:

$$ Accumulation = In - Out + Generation - Consumption $$

Let's apply this to our two components, A and B.

--- 
### Material Balance on Component A (Reactant)

*   **Accumulation:** The rate of change of moles of A in the tank. This is $\frac{d(V C_A)}{dt}$.
*   **In:** Molar flow of A into the tank. This is $F C_{A,in}$.
*   **Out:** Molar flow of A out of the tank. This is $F C_A$ (because the tank is perfectly mixed).
*   **Generation:** $0$. A is consumed, not generated.
*   **Consumption:** The rate at which A reacts away. The rate is $kC_A$ (moles per liter per minute), so the total consumption in the tank volume $V$ is $k C_A V$.

Putting it all together (and using $F$ for flow rate):
$$ \frac{d(V C_A)}{dt} = F C_{A,in} - F C_A - k C_A V $$

Since volume $V$ is constant, we can take it out of the derivative and divide the entire equation by $V$ to get our final ODE for $C_A$:
$$ \frac{dC_A}{dt} = \frac{F}{V} (C_{A,in} - C_A) - k C_A $$

--- 
### Material Balance on Component B (Product)

*   **Accumulation:** $\frac{d(V C_B)}{dt}$.
*   **In:** $F C_{B,in}$ (which is 0 in our case).
*   **Out:** $F C_B$.
*   **Generation:** The rate at which B is created by the reaction. From the stoichiometry $A \rightarrow B$, one mole of B is created for every mole of A consumed. So, the generation rate is $k C_A V$.
*   **Consumption:** $0$. B is a product.

Putting it all together:
$$ \frac{d(V C_B)}{dt} = F C_{B,in} - F C_B + k C_A V $$

Again, dividing by the constant volume $V$, we get our final ODE for $C_B$:
$$ \frac{dC_B}{dt} = \frac{F}{V} (C_{B,in} - C_B) + k C_A $$

## Part 4: Implementing the Model in Python

We have our system of two ODEs. Now we need to translate them into a Python function. The ODE solvers in `SciPy` require a function that takes the current time `t` and a list/vector of the state variables (in our case, the concentrations $[C_A, C_B]$) and returns a list of their derivatives $[dC_A/dt, dC_B/dt]$.

In [ ]:
def cstr_model(t, C):
    """
    Defines the system of ODEs for the CSTR based on material balances.
    
    Args:
        t (float): Current time. Required by the solver, but not used in our equations
                   because our parameters (k, V, flow_rate) are constant.
        C (list or array): A list containing the current concentrations [C_A, C_B].

    Returns:
        list: A list of the calculated derivatives [dC_A/dt, dC_B/dt].
    """
    # Unpack the concentration vector for clarity
    C_A, C_B = C
    
    # Implement the material balance equations we derived in Part 3
    dC_A_dt = (flow_rate / V) * (C_A_in - C_A) - k * C_A
    dC_B_dt = (flow_rate / V) * (C_B_in - C_B) + k * C_A
    
    return [dC_A_dt, dC_B_dt]

print("CSTR model function is created and ready for the solver.")

## Part 5: Simulating the Process Dynamics

With our model defined, we can now perform the simulation. It's often useful to first calculate the final, steady-state condition analytically. This gives us a target to check if our dynamic simulation is behaving correctly.

### 5a. Analytical Steady-State Calculation

At steady state, there is no more accumulation, so all time derivatives are zero ($\frac{dC}{dt} = 0$).

For component A:
$$ 0 = \frac{F}{V} (C_{A,in} - C_{A,ss}) - k C_{A,ss} $$

For component B:
$$ 0 = \frac{F}{V} (C_{B,in} - C_{B,ss}) + k C_{A,ss} $$

We can solve these simple algebraic equations for the steady-state concentrations ($C_{A,ss}$ and $C_{B,ss}$).

In [ ]:
# Solve for C_A_ss from the first equation
C_A_ss = (flow_rate * C_A_in) / (flow_rate + k * V)

# Substitute C_A_ss into the second equation and solve for C_B_ss
C_B_ss = (flow_rate * C_B_in + k * V * C_A_ss) / flow_rate

print("--- Analytical Steady-State ---")
print(f"Steady-State Concentration of A: {C_A_ss:.4f} mol/L")
print(f"Steady-State Concentration of B: {C_B_ss:.4f} mol/L")

### 5b. Running the Dynamic Simulation

Now, we'll solve the ODEs from our initial conditions over a specified time period.

In [ ]:
# Define the time span for the simulation.
# A good rule of thumb is to simulate for several residence times.
# The residence time (tau) is the average time a fluid particle spends in the reactor: tau = V / F
tau = V / flow_rate
t_final = 8 * tau  # Let's simulate for 8 residence times to be sure we reach steady state
t_span = (0, t_final)

# Pack the initial conditions into a single vector, in the same order as our ODE function
C_initial = [C_A0, C_B0]

# --- Call the ODE solver ---
# `solve_ivp` handles all the numerical integration steps.
# `dense_output=True` creates an interpolation of the solution, so we can get smooth plots.
solution = solve_ivp(cstr_model, t_span, C_initial, dense_output=True)

# For plotting, we'll create 200 evenly spaced time points across our simulation span
t_plot = np.linspace(t_span[0], t_span[1], 200)
# And we use the solver's dense output to find the solution at exactly these points
C_plot = solution.sol(t_plot)

print(f"Simulation complete. Solved from t=0 to t={t_final} minutes.")
print("The results for C_A and C_B over time are now stored and ready for plotting.")

## Part 6: Analyzing the Results - The Engineer's Dashboard

A simulation is only useful if we can interpret the results. Plotting is the best way to understand the dynamic behavior of the system.

In [ ]:
# Create the plot
plt.style.use('seaborn-v0_8-whitegrid') # Use a nice plot style
fig, ax = plt.subplots(figsize=(12, 7)) # Create a figure and axes for the plot

# Plot the dynamic concentration of A
ax.plot(t_plot, C_plot[0], label='$C_A$ (dynamic)', color='royalblue', linewidth=2.5)
# Plot the dynamic concentration of B
ax.plot(t_plot, C_plot[1], label='$C_B$ (dynamic)', color='firebrick', linewidth=2.5)

# Plot the steady-state concentrations as dashed lines for comparison
ax.axhline(y=C_A_ss, color='royalblue', linestyle='--', label='$C_{A,ss}$')
ax.axhline(y=C_B_ss, color='firebrick', linestyle='--', label='$C_{B,ss}$')

# Add labels, title, and legend for a professional-looking plot
ax.set_title('Dynamic CSTR Concentration Profile', fontsize=18, weight='bold')
ax.set_xlabel('Time (minutes)', fontsize=14)
ax.set_ylabel('Concentration (mol/L)', fontsize=14)
ax.legend(fontsize=12)
ax.set_xlim(0, t_final)
ax.set_ylim(0, C_A_in * 1.1)

# Display the plot
plt.show()

### Interpreting the Plot

This graph tells the complete story of our reactor's startup:
*   **Concentration of A (Blue):** It starts at its initial value ($1.0$ mol/L) and immediately begins to decrease. This is because as soon as it enters the reactor, it starts being consumed by the reaction. It eventually levels off at the calculated steady-state value ($C_{A,ss}$).
*   **Concentration of B (Red):** It starts at zero and its concentration rises. This is because it is being generated by the reaction. It also levels off at its calculated steady-state value ($C_{B,ss}$).
*   **The Agreement:** The fact that the dynamic curves level off exactly where our analytical steady-state lines are gives us high confidence that our model and our simulation are correct!
*   **Time to Steady State:** We can see that it takes approximately 400-500 minutes for the system to become stable. This is critical information for plant operations.

## Part 7: Becoming the Process Engineer - "What If?" Scenarios

Now it's your turn! The real power of simulation is the ability to ask "what if" without expensive and time-consuming plant trials. Try to answer the following questions by modifying the parameters in **Part 2** and re-running the notebook (`Cell > Run All`).

#### Challenge 1: A More Effective Catalyst
Your R&D team has developed a new catalyst that doubles the reaction rate. What is the new reaction rate constant `k`? Change it in Part 2. 
*Before you run*, predict the outcome: Will the steady-state concentration of A be higher or lower? What about B? Will the reactor reach steady state faster?

#### Challenge 2: Increasing Throughput
Management wants to increase production by doubling the `flow_rate`. Make this change.
*Predict*: How will this affect the residence time? Will the reactor have more or less time to react? What will this do to the concentration of product B at the outlet (i.e., its conversion)?

#### Challenge 3 (Advanced): A Second-Order Reaction
Let's model a different reaction: **$2A \rightarrow B$**. The rate law is now second-order: **$r = k C_A^2$**. You will need to modify the material balances in the `cstr_model` function in **Part 4**. 
*Hint*: The consumption term for A in the ODE is now `-2 * k * C_A**2` (the 2 is for stoichiometry). The generation term for B is `+1 * k * C_A**2`. Make these changes and see how the dynamics change.